In [253]:
import numpy as np
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor
import networkx as nx

In [254]:
Xs = ["X1", "X2", "X3", "X4", "X5"]
G = FactorGraph()
G.add_nodes_from(Xs)

K = 2 

# phi = {
#     "X1": np.array([1.0, 1.0]),
#     "X2": np.array([0.8, 1.2]),
#     "X3": np.array([1.0, 1.0]),
#     "X4": np.array([0.4, 2.0]),  
#     "X5": np.array([1.0, 1.0]),
# }

psi = {
    ("X1", "X2"): np.array([[3.0, 0.5], [0.5, 3.0]]),
    ("X2", "X3"): np.array([[1.0, 1.0], [1.0, 1.0]]),
    ("X2", "X4"): np.array([[3.0, 0.5], [0.5, 3.0]]),
    ("X4", "X5"): np.array([[0.25, 2.0], [2.0, 0.25]]),
}

# unary_nodes = {}
# for X in Xs:
#     f = DiscreteFactor([X], [K], phi[X])  # values length K
#     G.add_factors(f)
#     G.add_edge(X, f)
#     unary_nodes[X] = f

pair_nodes = {}
for (U, V), table in psi.items():
    f = DiscreteFactor([U, V], [K, K], table.ravel(order="C"))  # rows=U, cols=V
    G.add_factors(f)
    G.add_edge(U, f)
    G.add_edge(V, f)
    pair_nodes[(U, V)] = f

print("Variables:", [n for n in G.nodes() if not isinstance(n, DiscreteFactor)])
print("Factor count:", len(getattr(G, "get_factors", lambda: G.factors)()))
# Optional inspect: scopes of pairwise factors
print("Pairwise scopes:", [list(f.scope()) for f in pair_nodes.values()])

Variables: ['X1', 'X2', 'X3', 'X4', 'X5']
Factor count: 4
Pairwise scopes: [['X1', 'X2'], ['X2', 'X3'], ['X2', 'X4'], ['X4', 'X5']]


In [255]:
bfs_edges = list(nx.bfs_edges(G, source="X2"))
node_edge_list = [
    np.array(edge_0.scope()) for edge_0, edge_1 in bfs_edges if isinstance(edge_1, str)
]
factor_edge_list = [
    np.array(edge_0) for edge_0, edge_1 in bfs_edges if isinstance(edge_1, str)
]
node_factor_list = [np.array([edge_0, edge_1]) for edge_0, edge_1 in bfs_edges]
print("BFS Edges:", node_factor_list)

BFS Edges: [array(['X2', <DiscreteFactor representing phi(X1:2, X2:2) at 0x11da0da50>],
      dtype=object), array(['X2', <DiscreteFactor representing phi(X2:2, X3:2) at 0x11da0fc50>],
      dtype=object), array(['X2', <DiscreteFactor representing phi(X2:2, X4:2) at 0x11da0c050>],
      dtype=object), array([<DiscreteFactor representing phi(X1:2, X2:2) at 0x11da0da50>, 'X1'],
      dtype=object), array([<DiscreteFactor representing phi(X2:2, X3:2) at 0x11da0fc50>, 'X3'],
      dtype=object), array([<DiscreteFactor representing phi(X2:2, X4:2) at 0x11da0c050>, 'X4'],
      dtype=object), array(['X4', <DiscreteFactor representing phi(X4:2, X5:2) at 0x11da0ec10>],
      dtype=object), array([<DiscreteFactor representing phi(X4:2, X5:2) at 0x11da0ec10>, 'X5'],
      dtype=object)]


In [256]:
observed_set = {"X2"}
ordered_edge_list = []
for item in node_edge_list:
    if item[1] in observed_set:
        ordered_edge_list.append(np.array([item[1], item[0]]))
    else:
        ordered_edge_list.append(np.array([item[0], item[1]]))

print(ordered_edge_list)

[array(['X2', 'X1'], dtype='<U2'), array(['X2', 'X3'], dtype='<U2'), array(['X2', 'X4'], dtype='<U2'), array(['X4', 'X5'], dtype='<U2')]


In [257]:
source_of_edges = {edge[0] for edge in ordered_edge_list}
target_of_edges = {edge[1] for edge in ordered_edge_list}

leaf_nodes = [t for t in target_of_edges if t not in source_of_edges]

print("Leaf nodes:", leaf_nodes)

Leaf nodes: [np.str_('X1'), np.str_('X3'), np.str_('X5')]


In [258]:
source_of_edges = {edge[0] for edge in node_factor_list}
target_of_edges = {edge[1] for edge in node_factor_list}

leaf_nodes = [t for t in target_of_edges if t not in source_of_edges]

print("Leaf nodes:", leaf_nodes)

Leaf nodes: ['X3', 'X5', 'X1']


In [259]:
from_leaves_to_root = node_factor_list[::-1]
from_leaves_to_root

[array([<DiscreteFactor representing phi(X4:2, X5:2) at 0x11da0ec10>, 'X5'],
       dtype=object),
 array(['X4', <DiscreteFactor representing phi(X4:2, X5:2) at 0x11da0ec10>],
       dtype=object),
 array([<DiscreteFactor representing phi(X2:2, X4:2) at 0x11da0c050>, 'X4'],
       dtype=object),
 array([<DiscreteFactor representing phi(X2:2, X3:2) at 0x11da0fc50>, 'X3'],
       dtype=object),
 array([<DiscreteFactor representing phi(X1:2, X2:2) at 0x11da0da50>, 'X1'],
       dtype=object),
 array(['X2', <DiscreteFactor representing phi(X2:2, X4:2) at 0x11da0c050>],
       dtype=object),
 array(['X2', <DiscreteFactor representing phi(X2:2, X3:2) at 0x11da0fc50>],
       dtype=object),
 array(['X2', <DiscreteFactor representing phi(X1:2, X2:2) at 0x11da0da50>],
       dtype=object)]

In [260]:
factor_dict = {}
variable_dict = {}

In [261]:
def node_to_factor(node_source):
    node_value = 1
    for neighbor in G.neighbors(node_source):
        if neighbor in factor_dict.keys():
            node_value *= factor_dict[neighbor][
                node_source
            ]
    variable_dict[node_source] = node_value

In [262]:
def factor_to_node(node_source, node_target):
    sum_out = [v for v in node_source.variables if v != node_target]
    node_margine = node_source.marginalize(variables=sum_out, inplace=False).values

    for neighbor in G.neighbors(node_source):
        if neighbor in variable_dict.keys() and neighbor != node_target:
            node_margine *= variable_dict[neighbor]

    if node_source not in factor_dict:
        factor_dict[node_source] = {}
    factor_dict[node_source][node_target] = node_margine

In [263]:
def sum_product_algorithm(source):
    for node in from_leaves_to_root:
        if isinstance(node[1], DiscreteFactor):
            factor_to_node(node[1], node[0])
        else:
            node_to_factor(node[1])
    node_to_factor(source)
    return variable_dict, factor_dict


print(sum_product_algorithm("X2"))

({'X5': 1, 'X4': array([2.25, 2.25]), 'X3': 1, 'X1': 1, 'X2': array([55.125, 55.125])}, {<DiscreteFactor representing phi(X4:2, X5:2) at 0x11da0ec10>: {'X4': array([2.25, 2.25])}, <DiscreteFactor representing phi(X2:2, X4:2) at 0x11da0c050>: {'X2': array([7.875, 7.875])}, <DiscreteFactor representing phi(X2:2, X3:2) at 0x11da0fc50>: {'X2': array([2., 2.])}, <DiscreteFactor representing phi(X1:2, X2:2) at 0x11da0da50>: {'X2': array([3.5, 3.5])}})
